<h2>Computational Essay: Lydbølge på mikroskopisk nivå<h2>

Jeg skal utforske om det er mulig å simmulere hvordan en lydbølge sprer seg i rommet på mikroskopisk skala. Dette planlegger jeg å gjøre ved å simmulere partikkelkolisjoner, og se om ved riktig initialbetingelser om vi kan observere en bølgeform spre seg i rommet. Jeg vil prøve å se om det i det hele tatt blir mulig å se en bølgeform, og hvis det går vil jeg se hvor mange partikler jeg trenger i rommet for å se en bølge. Med det kan jeg sammenligne resultatet med lufta her på jorda.

<h3>Partikkelkollisjon<h3>


Her bruker jeg Lennard-Jones potensialet for å finne kraften hver partikkel gjør på hverandre. Lennard-Jones potensialet beskriver potensiell energi mellom to nøytralt ladde partikler. $\\$
Lennard-jones potensialet er gitt som: $$ V(r) = 4\epsilon \left( \left( \frac{\sigma}{r}\right)^{12} - \left( \frac{\sigma}{r}\right)^{6} \right)$$
Så vet vi at $F = -\nabla V$. For å forenkle alle utregninger jeg kommer til å gjøre så setter jeg $\sigma=\epsilon=m=1$, og da får vi at:
$$a = 24\left(\left( \frac{2}{|r|}\right)^{12}  - \left( \frac{1}{|r|}\right)^{6}\right)\frac{r}{|r|^2}$$
Da har vi alt vi trenger for å starte. Vi regner akselerasjonen alle partikler får fra sine nabopartikler, avhengig av distansen fra hverandre.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit
from tqdm import trange
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
np.random.seed(123)


In [2]:
%matplotlib notebook

Vi bruker velocity verlet som vår integrasjonsmetode i denne oppgaven

In [3]:
@njit
def calc_forces(r, N,n, cutoff):
    a = np.zeros((N, 2))
    u = 0
    for j in range(N):
        for k in range(j+1, N):
            if j != k:
                #let initial particles be unaffected by the impact of each other
                if k <= n  and j <= n:
                    pass
                else:
                    rvec = r[j] - r[k]
                    rvecnorm = np.linalg.norm(rvec)
                    #as rvecnorm increases the force decreases, so I put a cutoff at 3
                    rvecnorm2 = rvecnorm**2
                    if rvecnorm < cutoff and rvecnorm > 0:
                        a[j] += 24*(2/rvecnorm2**6 - 1/rvecnorm2**3)*rvec/rvecnorm2
                        a[k] -= 24*(2/rvecnorm2**6 - 1/rvecnorm2**3)*rvec/rvecnorm2
                        u += 4*(rvecnorm2**(-6) - rvecnorm2**(-3))
    return a,u

@njit
def integrate(r, v, a, dt, N,n, cutoff):
    aprev = a
    r += v*dt + 0.5*a*dt**2
    a,u = calc_forces(r, N,n, cutoff)
    v += 0.5*(a+aprev)*dt
    return r, v, a, u

Så lagde jeg en klasse for alt jeg kunne ønsket meg å gjøre. Jeg har lagt inn muligheten til å både putte partiklene tilfeldig rundt i rommet og uniformt. Jeg liker bedre å se på partiklene som blir plassert uniformt, fordi det blir litt mindre rot i integrasjonen når alle starter med lik distanse fra hverandre. Jeg gir alle partiklene en tilfeldig rettet hastighetsvektor ut ifra temperaturen som jeg satt til å være $20^{o}C$.

In [4]:

class SoundWave:
    def __init__(self,N,n,l,t,dt,T,random = True) -> None:
        '''
        N = number of particles
        n = number of particles entering
        l = length of box
        t = time
        dt = time step
        T = temperature
        random = True if initial positions are random, False if uniformly distributed. RN random is chaotic and not very interesting.
        '''
        self.N = int(N)
        self.n = int(n)
        self.l = l
        self.t = t
        self.dt = dt
        self.steps = int(t/dt)

        #at random
        if random == True:
            self.r0 = SoundWave.scatter_particles_random(N, l=20, r=0.6)
            self.r = np.zeros((self.steps,self.r0.shape[0] + self.n,2))
            self.r[0,self.n:] = self.r0

        #uniformly distributed
        else:
            self.r0 = SoundWave.scatter_particles(N,l)
            self.r = np.zeros((self.steps,self.r0.shape[0] + self.n,2))
            self.r[0,self.n:] = self.r0


        theta = np.linspace(0,2*np.pi,self.n)
        r0_impact = np.einsum('ij->ji',np.array([np.cos(theta),np.sin(theta)]))*0.5
        self.r[0,:self.n] = r0_impact
                
        
        self.v = np.zeros(self.r.shape)
        self.v0 = np.sqrt(3*T/120)
        self.v[0,self.n:] = np.random.normal(0, self.v0, size=(self.r0.shape[0], 2))

        v0_impact = r0_impact/np.linalg.norm(r0_impact)*self.v0*30

        self.v[0,:self.n] =  np.full((self.n, 2), v0_impact)
       

    def plot_start(self):
        '''Plot initial positions of particles'''
        plt.figure()
        l = self.l
        radius = 0.305
        plt.scatter(self.r[0,:,0],self.r[0,:,1],s=(radius / l) ** 2 * 1000)
        for i in range(self.n):
            plt.arrow(self.r[0,i,0],self.r[0,i,1],self.v[0,i,0]/20,self.v[0,i,1]/20, width = 0.05, color = 'r' )
        plt.xlabel('x[nm]')
        plt.ylabel('y[nm]')
        plt.title('Initial positions of particles')
        plt.grid()
        plt.show()

    def run(self):
        '''Run the simulation'''
        '''Lennard_jones potential:  a = 24/m*epsilon*(2*sigma**12/r**12 - sigma**6/r**6)*r/r**2.
        Natural units: epsilon = 1, sigma = 1, m = 1
        a = 24*(2/r**12 - 1/r**6)*r/r**2'''

        self.a = np.zeros(self.r.shape)
        self.u = np.zeros(self.steps)
        cutoff = 3#nm
        for i in trange(self.steps-1):
        
            self.r[i+1], self.v[i+1], self.a[i+1], self.u[i+1] = integrate(self.r[i], self.v[i], self.a[i], self.dt, self.r.shape[1],self.n, cutoff)

        return self.r,self.v

    def animate(self, sine = False):
        '''Only after "run" function'''

        #This function will animate the movement of the particles

        # create the figure and axes
        fig, ax = plt.subplots()
        ax.set_xlim(-self.l*.5, self.l*.5)
        ax.set_ylim(-self.l*.5, self.l*.5)
        ax.set_xlabel('x[nm]')
        ax.set_ylabel('y[nm]')
        ax.set_title('Animasjon av partikkelbevegelse')
        ax.set_aspect('equal')
        ax.grid()

        l = self.l
        radius = 0.305

        scatter = ax.scatter(self.r[0, :, 0], self.r[0, :, 1], s=(radius / l) ** 2 * 1000,linewidths=2)

        if sine == True:
            # Create a circle of points
            t = np.linspace(0, 2*np.pi, self.steps)
            x = np.cos(t)
            y = np.sin(t)


            # Initialize the line object with the circle points
            line, = ax.plot(x, y, '-', lw=2)
            k = 1
            v = 1
        
        def update(i):
            scatter.set_offsets(self.r[i])

            if sine == True:
                r_ = i/(self.steps)*self.l/2
                # Calculate the x- and y-coordinates of the wavefront
                R = np.sqrt(x**2 + y**2)
                
                y_new = r_ * np.sin(k * (R - v * t))
                x_new = r_ * np.sin(k * (R - v * t) -  np.pi/2)
                # Update the x- and y-coordinates of the circle points with the wavefront

                line.set_xdata(x + x_new)
                line.set_ydata(y + y_new)

                return scatter,line,
            else:
                return scatter,

        # create the animation
        anim = FuncAnimation(fig, update, frames=range(0, self.steps, 10), interval=50, blit=True)

        # show the animation
        plt.show()
    
    def EnergyPlot(self):
        '''Only after "run" function'''
        K = 0.5*np.sum(self.v**2,axis=(1,2))
        U = self.u
        E = K+U
        T = np.linspace(0,self.t,self.steps)
        
        plt.plot(T,K,label='Kinetisk energi')
        plt.plot(T,U,label='Potensiell energi')
        plt.plot(T,E,label='Total energi')
        plt.xlabel('Tid [s]')
        plt.ylabel('Energi [J]')
        plt.grid()
        plt.title('Energi')
        plt.legend()
        plt.show()
    
    def __str__(self) -> str:
        '''Prints a string with information about the simulation'''
        #calculate density
        m_H = 1.6737236e-27
        nm3_to_m3 = 1e-27
        total_mass = (self.N+self.n)**(3/2)*m_H
        volume = self.l**3*nm3_to_m3
        density = total_mass/volume
        return f'Box with {self.N} particles and {self.n} particles with inital velocity {self.v0:.2f} m/s. The box has a side length of {self.l} nm and a density of {density:.3f} kg/m^3 or {density/1.2:.3f} times the density of air on the surface of earth'

    def scatter_particles_random(N, l=20, r=0.5):
        """
        Scatter N points within a box of side length l, except for a 1x1 box around the origin.
        Points are scattered randomly with a minimum distance from any other point of 0.5.
        If there are too many points for this to be possible, the remaining particles are scattered outside of the box.
        """
        # Create an empty array to store the scattered points
        particles = np.empty((0, 2), dtype=float)

        # Scatter points within the box, excluding a 1x1 box around the origin
        while len(particles) < N:
            # Generate random coordinates within the box
            x = np.random.uniform(-l/2, l/2)
            y = np.random.uniform(-l/2, l/2)

            # Check if the point is outside the 1x1 box around the origin
            if abs(x) > 1 or abs(y) > 1:
                # Check if the point is at least 0.5 units away from any other point
                if np.all(np.linalg.norm(particles - np.array([x, y]), axis=1) >= r):
                    # Add the point to the list of scattered points
                    particles = np.vstack((particles, np.array([x, y])))
        # Return the scattered points
        return particles

    def scatter_particles(N,l):
        lb = -l*0.5
        ub = l*0.5
        sq_size = l*0.1
        # Calculate spacing between points based on total number of points and size of grid
        grid_size = int(np.sqrt(N)) + 1  # Calculate grid size based on square root of N
        spacing = (ub - lb - sq_size) / (grid_size - 1)  # Calculate spacing between points
        # Generate x and y values with equal spacing
        x = np.linspace(lb, ub, grid_size)
        y = np.linspace(lb, ub, grid_size)
        xx, yy = np.meshgrid(x, y)
        xx = xx.flatten()
        yy = yy.flatten()

        # Filter out points within the square around origin
        x_filtered = []
        y_filtered = []
        for i in range(len(xx)):
            if abs(xx[i]) > sq_size/2 or abs(yy[i]) > sq_size/2:
                x_filtered.append(xx[i])
                y_filtered.append(yy[i])
        x_filtered = x_filtered[:N]
        y_filtered = y_filtered[:N]
        return np.array([x_filtered, y_filtered]).T

<h3>Sammenlikning med sinusbølge radielt i rommet<h3>

For å se om simmulasjonen min kan stemme ønsker jeg å sammenligne det med en sinusbølge som strekker seg radielt ut i rommet.$\\$
Bølgen er beskrevet som $y(x,t) = A\sin(kx - 2\pi f t) = A\sin(kx - \omega t)$. Hvor $k=\omega /v$. Jeg holder meg til lave frekvenser og lave hastigheter, siden jeg allerede simmulerer nanometer skala så vil det være meningsløst å se på superhøye hastigheter. Så jeg tar det som en "saktefilm"

In [5]:
def animate_sine_wave(duration, sample_rate, radius):
    fig, ax = plt.subplots()
    ax.set_xlim(-radius*1.1, radius*1.1)
    ax.set_ylim(-radius*1.1, radius*1.1)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('Sound wave radiating from a point')
    ax.set_aspect('equal')
    ax.grid()

    # Create a circle of points
    t = np.linspace(0, 2*np.pi, sample_rate*duration, endpoint=False)
    x = np.cos(t)
    y = np.sin(t)


    # Initialize the line object with the circle points
    line, = ax.plot(x, y, '-', lw=2)
    k = 1
    v = 1

    def update(i):
        # Calculate the radius of the wavefront at this time point
        r = i/(sample_rate*duration)*radius
        # Calculate the x- and y-coordinates of the wavefront
        R = np.sqrt(x**2 + y**2)
        
        y_new = r * np.sin(k * R - v * t)
        x_new = r * np.sin(k * R - v * t -  np.pi/2)
        # Update the x- and y-coordinates of the circle points with the wavefront

        line.set_xdata(x + x_new)
        line.set_ydata(y + y_new)
        return line,

    # Create the animation
    anim = FuncAnimation(fig, update, frames=range(0,sample_rate*duration,10), interval=20, blit=True)

    return anim
duration = 1 # seconds
sample_rate = 4000 # samples per second
anim = animate_sine_wave(duration, sample_rate, 20)

# Show the animation
plt.show()


<IPython.core.display.Javascript object>

Nå prøver vi å kjøre en animasjon med mange partikler for å se om det går an

In [6]:
N = 600
n = N/10
l = 20 #nm
K = 293 #20C in Kelvin
sound_600 = SoundWave(N,n,l,0.5,5e-4,K,random=False)
sound_600.plot_start()
r,v = sound_600.run()
sound_600.EnergyPlot()
print(sound_600)
# sound.animate(sine = False)
# sound.animate(sine = True)

<IPython.core.display.Javascript object>

100%|██████████| 999/999 [02:14<00:00,  7.42it/s]

Box with 600 particles and 60 particles with inital velocity 2.71 m/s. The box has a side length of 20 nm and a density of 3.547 kg/m^3 or 2.956 times the density of air on the surface of earth


Nå med litt ferre partikler

In [7]:
N = 20
n = N/10
l = 20 #nm
K = 293 #20C in Kelvin
sound = SoundWave(N,n,l,0.5,5e-4,K,random=False)
sound.plot_start()
r,v = sound.run()
sound.EnergyPlot()
print(sound)
# sound.animate(sine = False)
# sound.animate(sine = True)

<IPython.core.display.Javascript object>

100%|██████████| 999/999 [00:00<00:00, 2846.23it/s]

Box with 20 particles and 2 particles with inital velocity 2.71 m/s. The box has a side length of 20 nm and a density of 0.022 kg/m^3 or 0.018 times the density of air on the surface of earth


In [8]:
a0 =sound_600.animate(sine = False)
a1 =sound_600.animate(sine = True)
a2 =sound.animate(sine = False)
a3 =sound.animate(sine = True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

C:\Users\oskar\AppData\Roaming\Python\Python39\site-packages\matplotlib\animation.py:879: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you output the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<h3>Konklusjon<h3>

Når jeg setter inn 600 partikler i boksen, ser vi veldig tydelig at det er en bølgeform som sprer seg i rommet og den er veldig lett å sammenligne med sinusbølgen. Men da får vi et lufttetthet mye høyere enn det vi har her på jorda, så da tar vi det som en selvfølge at en lydbølge vil kunne formes. $\\$
Jeg merket at rundt 200 partikler i rommet, som tilsvarer omtrent 0.55 ganger lufttettheten vi har her på jorda, så begynte det å bli vanskelig å se en lydbølge forme.  $\\$ Jeg merket at det vil være vanskelig å gi et fasitsvar på hvor høy luftettheten må være for at en lydbølge skal kunne bevege seg, men det var interessant å se en bølgeform spre. Jeg prøvde å sammenlikne med sinusbølge i bakgrunnen for å se hvordan det egentlig skal spre seg. Ved et høyt antall partikler så det ut til å funke ganske bra, men igjen så var det vanskelig å finne en nedre grense på hvor mange partikler vi trenger 